In [1]:
import pandas as pd
from pandas import Series, DataFrame

In [2]:
from pandas import read_html

In [21]:
url = 'https://en.wikipedia.org/wiki/NFL_win-loss_records'

In [25]:
import urllib3,certifi
 
 #Force certificate check and use certifi to handle the certificate. 
https = urllib3.PoolManager( cert_reqs='CERT_REQUIRED',
ca_certs=certifi.where(),)   
url = https.urlopen('GET','https://en.wikipedia.org/wiki/NFL_win-loss_records') 

#Then you parse the html as usual 
dframe_list = pd.read_html(url.data)
#dframe_list = pd.io.html.read_html(url) 

In [28]:
dframe_list[1]

,0,1,2,3,4,5,6,7,8
0,Rank,Team,Won,Lost,Tied,Pct.,First NFL Season,Total Games,Division
1,1,Dallas Cowboys,502,374,6,.573,1960,882,NFC East
2,2,Green Bay Packers,737,562,37,.565,1921,1336,NFC North
3,3,Chicago Bears,749,579,42,.562,1920,1370,NFC North
4,4,Miami Dolphins,445,351,4,.559,1966,800,AFC East
5,5,New England Patriots[b],489,386,9,.558,1960,884,AFC East
6,6,Minnesota Vikings,470,390,10,.546,1961,870,NFC North
7,7,Baltimore Ravens,190,161,1,.541,1996,352,AFC North
8,8,New York Giants,687,585,33,.539,1925,1305,NFC East
9,9,Denver Broncos,470,404,10,.537,1960,884,AFC West


In [36]:
import urllib.request
import ssl
import time
from datetime import timedelta
from bs4 import BeautifulSoup
from pandas import *
from dateutil.parser import parse

#date to second
def ISOString2Time(s):
    period=parse(str(s))+timedelta(days=1)
    d = datetime.strptime(str(period), "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(d.timetuple()))

Name = "^GSPC"
FirstDate = 20170124
EndDate = 20170224


def yahoo_finance(Name, FirstDate, EndDate):
    period1 = ISOString2Time(FirstDate)
    period2 = ISOString2Time(EndDate)
    url = "https://finance.yahoo.com/quote/" + Name + "/history?period1=" + str(period1) + "&period2=" + \
          str(period2) + "&interval=1d&filter=history&frequency=1d"
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request, context=ssl._create_unverified_context())
    data = response.read()
    soup = BeautifulSoup(data, "lxml")
    table = soup.find('table')
    x = len(table.findAll('tr')) - 1
    if x != 1:
        for row in range(1, x):
            col = table.findAll('tr')[row].findAll('td')
            if len(col) == 7:
                Date = col[0].getText()
                Open = col[1].getText()
                High = col[2].getText()
                Low = col[3].getText()
                Close = col[4].getText()
                Adj_Close = col[5].getText()
                Volume = col[6].getText()
                Data = DataFrame([Date, Open, High, Low, Close, Adj_Close, Volume],
                                 index=["Date", "Open", "High", "Low", "Close", "Adj_Close", "Volume"]).T
                if row == 1:
                    Sheet = Data
                else:
                    Sheet = Sheet.append(Data, ignore_index=True)
            continue
        return Sheet
    else:
        print("There is no " + Name + " index or date")


AGG = yahoo_finance(Name="AGG", FirstDate=20170124, EndDate=20170224)
IVV = yahoo_finance(Name="IVV", FirstDate=20170124, EndDate=20170224)
IEV = yahoo_finance(Name="IEV", FirstDate=20170124, EndDate=20170224)
HYB = yahoo_finance(Name="HYB", FirstDate=20170124, EndDate=20170224)
EMB = yahoo_finance(Name="EMB", FirstDate=20170124, EndDate=20170224)
JPY = yahoo_finance(Name=".SI", FirstDate=20170124, EndDate=20170224)
IAU = yahoo_finance(Name="IAU", FirstDate=20170124, EndDate=20170224)
# AGG IVV IEV HYB EMB JPY IAU
'''
import time
import datetime
timeStamp = 1511452800
dateArray = datetime.datetime.utcfromtimestamp(timeStamp)
otherStyleTime = dateArray.strftime("%H:%M:%S")
period1 = str(datetime.datetime(2017,11,24))[0:11] + otherStyleTime
print(int(time.mktime(time.strptime(period1, "%Y-%m-%d %H:%M:%S"))))
print(period1)
print(dateArray)
'''

# second to date
def Time2ISOString(s):

    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(float(s)))


c = Time2ISOString(1511452800)

print(c)



There is no .SI index or date
2017-11-24 00:00:00


In [37]:
print(AGG)

            Date    Open    High     Low   Close Adj_Close     Volume
0   Feb 24, 2017  108.88  109.06  108.82  109.00    106.51  2,670,900
1   Feb 23, 2017  108.56  108.65  108.53  108.60    106.12  2,169,000
2   Feb 22, 2017  108.54  108.55  108.20  108.42    105.94  2,655,500
3   Feb 21, 2017  108.24  108.45  108.22  108.35    105.87  1,980,300
4   Feb 17, 2017  108.37  108.45  108.32  108.36    105.88  2,228,100
5   Feb 16, 2017  108.01  108.28  108.01  108.12    105.65  5,257,400
6   Feb 15, 2017  107.92  108.00  107.86  107.89    105.42  2,138,900
7   Feb 14, 2017  108.30  108.30  107.91  108.06    105.59  2,809,500
8   Feb 13, 2017  108.34  108.35  108.23  108.30    105.82  2,189,800
9   Feb 10, 2017  108.26  108.47  108.26  108.46    105.98  2,365,300
10  Feb 09, 2017  108.61  108.67  108.43  108.48    106.00  2,736,000
11  Feb 08, 2017  108.61  108.83  108.61  108.74    106.25  3,873,200
12  Feb 07, 2017  108.30  108.58  108.25  108.43    105.95  2,659,100
13  Feb 06, 2017  10